In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
image_size = (512, 512)
hydro_dir = r"mmflood/mmflood/output/Hydro_Standardized"
dem_dir = r"mmflood/mmflood/output/DEM_Standardized"
s1_raw_dir = r"mmflood/mmflood/output/S1_RGB_Img_Standardized"
mask_dir = r"mmflood/mmflood/output/Mask_Standardized"

In [3]:
hydro_files = sorted(os.listdir(hydro_dir))
dem_files = sorted(os.listdir(dem_dir))
s1_raw_files = sorted(os.listdir(s1_raw_dir))
mask_files = sorted(os.listdir(mask_dir))

common_files = set(hydro_files) & set(dem_files) & set(s1_raw_files) & set(mask_files)
common_files = sorted(list(common_files))

In [4]:
def load_common_images(image_paths, common_files):
    images = []
    for file in common_files:
        img_path = os.path.join(image_paths, file)
        img = load_img(img_path, target_size=image_size)
        img = img_to_array(img) / 255.0
        images.append(img)
    return np.array(images)

In [5]:
hydro_data = load_common_images(hydro_dir, common_files)
dem_data = load_common_images(dem_dir, common_files)
s1_raw_data = load_common_images(s1_raw_dir, common_files)
mask_data = load_common_images(mask_dir, common_files)

In [6]:
mask_data = (mask_data > 0.5).astype(np.uint8)

In [7]:
X = np.concatenate([hydro_data, dem_data, s1_raw_data], axis=-1)
y = mask_data

In [8]:
print(f"Hydro Data Shape: {hydro_data.shape}")
print(f"DEM Data Shape: {dem_data.shape}")
print(f"S1 Raw Data Shape: {s1_raw_data.shape}")
print(f"Mask Data Shape: {mask_data.shape}")
print(f"Final Input Shape (X): {X.shape}")

Hydro Data Shape: (1012, 512, 512, 3)
DEM Data Shape: (1012, 512, 512, 3)
S1 Raw Data Shape: (1012, 512, 512, 3)
Mask Data Shape: (1012, 512, 512, 3)
Final Input Shape (X): (1012, 512, 512, 9)


In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=176)
print(f"Training Data Shape: {X_train.shape}, Validation Data Shape: {X_val.shape}")

Training Data Shape: (809, 512, 512, 9), Validation Data Shape: (203, 512, 512, 9)


In [10]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, Concatenate, Activation

In [11]:
input_layer = Input(shape=(512, 512, 9))

In [12]:
base_model = ResNet50(include_top=False, weights=None, input_tensor=input_layer)

In [13]:
x = base_model.output  
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D(size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D(size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D(size=(2, 2))(x)
x = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)  

In [14]:
model = Model(inputs=input_layer, outputs=x)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [15]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 9)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 518, 518,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 9)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 256, 256,  │     28,288 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 256, 256,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 256, 256,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 258, 258,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 128, 128,  │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 128, 128,  │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 128, 128,  │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 128, 128,  │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 128, 128,  │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 128, 128,  │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 128, 128,  │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 128, 128,  │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 128, 128,  │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 128, 128,  │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 128, 128,  │      1,024 │ conv2_block1_3_c

 Total params: 28,694,273 (109.46 MB)

 Trainable params: 28,641,153 (109.26 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [17]:
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size=4, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.X))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        X_batch = np.array([self.X[k] for k in indexes])  
        y_batch = np.array([self.y[k] for k in indexes])  

        return X_batch, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [35]:
import cv2

y_train_resized = np.array([cv2.resize(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), (128, 128)) for mask in y_train])
y_train_resized = np.expand_dims(y_train_resized, axis=-1)  

y_val_resized = np.array([cv2.resize(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), (128, 128)) for mask in y_val])
y_val_resized = np.expand_dims(y_val_resized, axis=-1)  

In [41]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense
from tensorflow.keras.utils import Sequence

In [43]:
def preprocess_masks(masks):
    masks_resized = np.array([cv2.resize(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), (128, 128)) for mask in masks])
    masks_resized = np.expand_dims(masks_resized, axis=-1)  
    return masks_resized.astype("float32") / 255.0  

In [45]:
y_train_resized = preprocess_masks(y_train)
y_val_resized = preprocess_masks(y_val)

In [47]:
tf.keras.backend.clear_session()

In [49]:
def build_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(128, 128, 3)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        UpSampling2D((2,2)),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        UpSampling2D((2,2)),
        Conv2D(1, (1,1), activation='sigmoid')  # Output (128,128,1) for binary segmentation
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [51]:
model = build_model()

C:\Users\dhruv\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [53]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 128, 128, 1)    │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 167,105 (652.75 KB)

 Trainable params: 167,105 (652.75 KB)

 Non-trainable params: 0 (0.00 B)

In [113]:
import cv2
import numpy as np
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size=4, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.X))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.floor(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        X_batch = np.array([self.X[i] for i in batch_indexes])
        y_batch = np.array([self.y[i] for i in batch_indexes])


        y_batch_resized = np.array([cv2.resize(mask, (512, 512), interpolation=cv2.INTER_NEAREST) for mask in y_batch])
        y_batch_resized = np.expand_dims(y_batch_resized, axis=-1)

        if X_batch.shape[-1] > 3:
            X_batch = X_batch[:, :, :, :3]  

        return X_batch, y_batch_resized

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [115]:
train_generator = DataGenerator(X_train, y_train_resized, batch_size=4)
val_generator = DataGenerator(X_val, y_val_resized, batch_size=4)

In [117]:
x_batch, y_batch = train_generator[0]
print("Train batch shape:", x_batch.shape, y_batch.shape)  

Train batch shape: (4, 512, 512, 3) (4, 512, 512, 1)


In [127]:
model = build_model()

C:\Users\dhruv\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [131]:
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 664s 3s/step - accuracy: 0.5879 - loss: 0.0113 - val_accuracy: 0.5760 - val_loss: 0.0117
Epoch 2/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 692s 3s/step - accuracy: 0.5854 - loss: 0.0114 - val_accuracy: 0.5752 - val_loss: 0.0115
Epoch 3/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 662s 3s/step - accuracy: 0.5902 - loss: 0.0111 - val_accuracy: 0.5774 - val_loss: 0.0112
Epoch 4/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 701s 3s/step - accuracy: 0.5840 - loss: 0.0116 - val_accuracy: 0.5757 - val_loss: 0.0115
Epoch 5/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 535s 3s/step - accuracy: 0.5902 - loss: 0.0110 - val_accuracy: 0.5761 - val_loss: 0.0121
Epoch 6/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 571s 3s/step - accuracy: 0.5846 - loss: 0.0112 - val_accuracy: 0.5753 - val_loss: 0.0113
Epoch 7/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 563s 3s/step - accuracy: 0.5853 - loss: 0.0110 - val_accuracy: 0.5768 - val_loss: 0.0112
Epoch 8/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 572s 3s/step - accuracy: 0.5874 - loss: 0.0110 - val_accu

In [133]:
model.save("apun_ka_flood_prediction_model_1.h5") 

In [121]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)

X_train shape: (809, 512, 512, 3)
X_val shape: (203, 512, 512, 3)


In [123]:
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

y_train shape: (809, 512, 512, 3)
y_val shape: (203, 512, 512, 3)


In [125]:
X_batch, y_batch = train_generator[0]  # Try fetching first batch
print(X_batch.shape, y_batch.shape)

(4, 512, 512, 3) (4, 512, 512, 1)
